In [40]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqUtils import ProtParam
import os

In [41]:
file_path = "files/sprot_human.fasta"

In [33]:
#sequences = [i for i in SeqIO.parse(file_path,'fasta')]
#len(sequences)

In [34]:
def assignment(file_path, enzyme_rule, NTT, missed_cleavages, peptide_length):
    """
    function description - later
    """
    peptides = []
    with open(file_path) as handle:
        for record in SeqIO.parse(handle, "fasta"):
            prot_seq = str(record.seq)
            prot_header = record.description

            for i in range(len(prot_seq)):
                for j in range(i + peptide_length[0], min(i + peptide_length[1] + 1, len(prot_seq))):
                    peptide_seq = prot_seq[i:j]
                    pep_len = len(peptide_seq)
                    pep_NTT = 0

                    if peptide_seq[0] not in enzyme_rule:
                        continue
                    if peptide_seq[-1] not in enzyme_rule:
                        continue
                    if pep_len < peptide_length[0] or pep_len > peptide_length[1]:
                        continue
                    for k in range(1, pep_len):
                        if peptide_seq[k-1] in enzyme_rule and peptide_seq[k] not in enzyme_rule:
                            pep_NTT += 1
                    if pep_NTT > NTT:
                        continue

                    for k in range(1, missed_cleavages+2):
                        for l in range(pep_len-k+1):
                            missed_sites = [pos for pos, char in enumerate(peptide_seq) if char not in enzyme_rule]
                            if l not in missed_sites:
                                pep_missed_cleavages = k-1
                                peptide_num = len(peptides) + 1
                                peptides.append({
                                    "protein_header": prot_header,
                                    "protein_seq": prot_seq,
                                    "peptide_number": peptide_num,
                                    "peptide_sequence": peptide_seq,
                                    "peptide_length": pep_len,
                                    "peptide_NTT": pep_NTT,
                                    "peptide_missed_cleavages": pep_missed_cleavages
                                })
    return peptides

In [47]:
def print_peptide_of_interest(peptides,cleavage_rules,min_length,max_length):
    peptides = assignment(file_path,cleavage_rules,min_length,max_length,[7,20])
    for i, peptide in enumerate(peptides):
        sequence = peptide['sequence']
        NTT = peptide['NTT']
        missed_cleavages = peptide['missed_cleavages']
        length = peptide['length']
        start = peptide['start'] + 1
        end = peptide['end'] + 1
        prefix = sequence[0]
        suffix = sequence[-1]
        print(f"{i+1}. ({prefix}){sequence[1:-1]}({suffix}), {end-start}, {NTT}, {missed_cleavages}")


In [46]:
cleavage_rules = [["R", "K"], "C"]
min_length = 2
max_length = 20

print_peptide_of_interest(file_path, cleavage_rules, min_length, max_length)

AttributeError: 'list' object has no attribute 'id'

In [38]:
# Saving the Output instead of just printing it
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
output_file = 'files/output.fasta'
def save_peptide_of_interest(protein, cleavage_rules, min_length, max_length, output_file):
    peptides = assignment(file_path, cleavage_rules, min_length, max_length, [7,20])
    
    records = []
    for i, peptide in enumerate(peptides):
        sequence = peptide['sequence']
        NTT = peptide['NTT']
        missed_cleavages = peptide['missed_cleavages']
        length = peptide['length']
        start = peptide['start'] + 1
        end = peptide['end'] + 1
        prefix = sequence[0]
        suffix = sequence[-1]
        description = f"({prefix}){sequence[1:-1]}({suffix}), {end-start}, {NTT}, {missed_cleavages}"
        record = SeqRecord(Seq(sequence), id=f"{protein.id}_{i+1}", description=description)
        records.append(record)
    
    SeqIO.write(records, 'files/output.fasta', "fasta")


In [39]:
#SeqIO.write(gene_of_interest,'Spring Semester/Proteomics Analysis/Home Work/output/output.xml','fasta')